# A/B Testing Motivation

We have a current webpage. We want to implement a new webpage which is desired to have 2% higher of conversion. To measure the efficiency of the B-webpage, we must have two groups:

        - Group A: Control group, which is the one we evaluate the conversion of A-webpage.
        - Group B: Treatment group, which is the one we evaluate the performance of the B-webpage.
        
Thus, we will compare the performance of each webpage. The test must be performed by taking a sample from the population. 

#### Execution

- The groups must be homogeneous between each other. We have to create 2 groups randomly.
- We have to assure that the difference between conversions are not from the randomness of the samples and the sample size. We may avoid this problem by using a statistical inference method to decide if whether the data can support a particular hypothesis.
- To construct the hypothesis:

        A: the current scenario (status quo), without modifications. (null hypothesis)
            
        B: the new scenario, with some new implementation or modification. (alternative hypothesis)
        
#### Rational


**1.** We assume the **hypothesis A** as true. By doing this, we assume that there is no difference between the webpages. **(null hypothesis)**

**2.** By assuming **hypothesis A** as true and that there is no difference between the pages, if we observe a conversion value very far from each other, it may be caused by the experiment randomness.

**3.** If the probability (p-value) of happening the difference between the pages (null hypothesis) is very high: the randomness effect is so low that we may admit that the difference is caused by the pages. 

        That means: If the p-value is low, we reject the null hypothesis in favor of the alternative hypothesis. If it is high, we do not have enough evidence to reject the null hypothesis.
        
        The "high value" means higher than an alpha value (confidence value) we define. Usually, alpha is 0.05. Alpha is the maximum of randomness we admit.
        
**The main rational is:** if the probability **(p-value)** of happening the null hypothesis is higher than the maximum randomness **(alpha)** we admitted, therefore the null hypothesis is "more important" than randomness and we cannot reject null hypothesis - because we have low effect of randomness. But if the p-value is lower than the maximum randomness we admitted, thus the randomness is "more important" than the null hypothesis and therefore we may reject the null hypothesis. 
        

# Libraries

In [26]:
import pandas as pd
import numpy as np
import math
import pingouin as pg
from statsmodels.stats import api as sms

# Loading Data

In [2]:
pwd

'/Users/lucasquemelli/Documents/repos/AB testing'

In [3]:
data = pd.read_csv("/Users/lucasquemelli/Documents/repos/AB testing/ab_data.csv")

In [4]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [5]:
data.shape

(294478, 5)

# 1. Experiment Design

Based on the bussiness, we must define the hypotheses and the type of the test. We cannot use both of the tests mentioned below, we must choose only one.

## 1.1. Formulation of Hypotheses

As we already defined above, the hypotheses are as follows:

**1. Hypothesis A**: we assume that there is no difference between the webpages. **(null hypothesis)**

**2. Hypothesis B:** we assume that there is significant difference between the webpages. **(alternative hypothesis)**

## 1.2. One-tailed Test

A one-tailed test, also known as a one-sided test, is a statistical test used to determine if the sample data falls into a specific direction from the hypothesized population parameter. In a one-tailed test, the alternative hypothesis specifies a particular direction of the effect or difference being tested.

For example, if we are testing whether a new drug increases the average lifespan of patients, the one-tailed test would focus on whether the drug increases lifespan without considering the possibility of it decreasing lifespan.

The decision to use a one-tailed test depends on the specific research question and the directional hypothesis being tested. One-tailed tests are often used when there is strong theoretical or practical justification to expect an effect in a specific direction.

It's important to note that one-tailed tests can be more powerful than two-tailed tests in detecting effects in the specified direction, but they also carry the risk of overlooking effects in the opposite direction. Therefore, researchers should carefully consider the choice of test based on the nature of the research question and the potential consequences of Type I and Type II errors.

- **Null hypothesis:** when the probability of happen the null hypothesis (p<sub>0</sub>) is equal to the probability of happen the alternative hypothesis (p).
## H<sub>0</sub>: p = p<sub>0</sub>

- **Alternative hypothesis:** when the probability of happen the alternative hypothesis (p) is higher the probability of happen the null hypothesis (p<sub>0</sub>).
## H<sub>1</sub>: p > p<sub>0</sub>

**ATTENTION:** to use this test we must aske ourselves if the effect can only goes to one direction. Could that only be higher? Could that only be lower?

## 1.3. Two-tailed Test

By comparing two webpages, the effect is not only one direction: the conversion may be higher or lower. At such a situation we use the two-tailed test.

A two-tailed test, also known as a two-sided test, is a statistical test used to determine if there is a significant difference between the observed sample data and the hypothesized population parameter in either direction. In a two-tailed test, the alternative hypothesis typically states that the parameter is not equal to a specific value, without specifying a particular direction of the effect.

For example, if we are testing whether a new teaching method affects test scores, the two-tailed test would check whether there is a significant difference in test scores, regardless of whether they increase or decrease.

The decision to use a two-tailed test is often made when there is no prior expectation or justification for a particular direction of the effect. It allows for a more conservative approach by considering the possibility of differences in both directions.

In a two-tailed test, the critical region is split into two equal parts, with rejection regions in both tails of the distribution. The test statistic is compared to the critical values corresponding to the desired level of significance, typically α (alpha), divided equally between the two tails.

It's important to note that two-tailed tests are generally less powerful than one-tailed tests for detecting effects in a specific direction. However, they provide a more comprehensive assessment of the data by considering the possibility of differences in both directions.

Researchers should carefully consider the choice of test based on the specific research question, the nature of the data, and any prior theoretical or empirical evidence regarding the direction of the effect.

- **Null hypothesis:** when the probability of happen the null hypothesis (p<sub>0</sub>) is equal to the probability of happen the alternative hypothesis (p).
## H<sub>0</sub>: p = p<sub>0</sub>

- **Alternative hypothesis:** when the probability of happen the alternative hypothesis (p) is different of the probability of happen the null hypothesis (p<sub>0</sub>).
## H<sub>1</sub>: p <> p<sub>0</sub>

## 1.4. Tests Parameters

After deciding between one-tailed or two-tailed test, we must define the parameters.

### 1.4.1. Confidence Level

The confidence level in an A/B test is the measure of certainty with which you can trust the results of the test. It indicates the probability that the observed difference between groups (for example, the control group and the test group) is true and not merely due to chance.

Typically, the confidence level is expressed as a percentage, such as 95% or 99%. A confidence level of 95%, for example, means there is a 95% probability that the observed difference between groups is statistically significant and not just a result of random variability in the data.

The higher the chosen confidence level, the more confident you can be in the results of the test. However, it's important to balance the confidence level with sample size and test sensitivity, as very high confidence levels may require very large samples to detect small differences.

In summary, the confidence level in an A/B test is a measure of the probability that the observed results are statistically significant and not due to chance, and it's crucial for assessing the validity and reliability of the test results.

In [6]:
confidence_level = 0.95

### 1.4.2. Significance Level

This is: 1 - confidence level. 

The significance level in an A/B test, often denoted as α (alpha), is the probability of incorrectly rejecting the null hypothesis when it is actually true. In other words, it represents the threshold for determining whether an observed difference between groups is statistically significant or simply due to random variability.

Typically, the significance level is set before conducting the test and is commonly chosen to be 0.05 (5%) or 0.01 (1%). This means that if the p-value calculated from the test is less than the significance level (e.g., p < 0.05), the null hypothesis is rejected, and the difference between groups is considered statistically significant.

Conversely, if the p-value is greater than the significance level, the null hypothesis is not rejected, and the observed difference is not deemed statistically significant.

Choosing the significance level involves a trade-off between Type I and Type II errors. A lower significance level (e.g., α = 0.01) reduces the probability of Type I errors (false positives), but increases the likelihood of Type II errors (false negatives). Conversely, a higher significance level (e.g., α = 0.10) decreases the chance of Type II errors but increases the risk of Type I errors.

     Type I error: we reject the null hypothesis by understanding the alternative hypothesis is better, but not.
                   This is the probability of observing an event that does not exist.
         
     Type II error: we do not reject the null hypothesis by understanding it is better, but not.
                    This is the probability of not observing an event that do exist.

In summary, the significance level in an A/B test determines the threshold for determining statistical significance, and it's crucial for interpreting the results and making decisions based on the test outcomes.

In [7]:
significance_level = 1 - confidence_level

### 1.4.3. Sample Size

There are some parameters we must understand before determining the sample size:

- **effect size:** difference between **control** and **treatment** = 13% - 15% = 2%. It is a measure of the magnitude of the difference or effect that you want to detect in the test. Indicates the practical importance of the difference between groups. We have to test the sample size in order to find the optimum effect size.

        effect_size = p1 - p2, where p1 is the current conversion and p2 is the new conversion
        p1 = 13% = 0.13
        p2 = 15% = 0.15, it is defined by the bussiness team
        
        Larger differences demand less data points. However, Smaller differences demand more data points.
        
Here we present 4 ways of calculating the effect size. The most famous family to calculate the effect size is D (difference). 

    1. Cohen's d: difference between the average of the samples (A and B) over global standard deviation.
    2. Glass's delta: same as Cohen's d, but over only the standard deviation of the control group.
                      Used when there is n-treatment groups (A, B, C,... n test)
    3. Hedge's g: it is a type of Cohen's d, but for sample size lower than 20 (n < 20). 
    4. Cohen's h: used when 2 samples are independent. 

1. Cohen's d:

        X: Represents the data from one group (e.g., control group).
        Y: Represents the data from another group (e.g., experimental group).
        
        Small Effect Size: d=0.20
        Medium Effect Size: d=0.50
        Large Effect Size: d=0.80

2. Glass's delta:

        X: Represents the data from one group.
        Y: Represents the data from another group.

3. Hedge's g:

        X: Represents the data from one group.
        Y: Represents the data from another group.
        
4. Cohen's h:

        X: Represents the proportion or rate in one group.
        Y: Represents the proportion or rate in another group.

- **Statistical Power:** It is the probability that a statistical test will detect a real existing effect. It reflects the test's ability to identify significant differences when they actually exist. It is around 80% (standard - better explanation in the next sections). 

        Type II error is: β = 1 - statistical_power

- **Significance Level:** It is the probability of making a Type I error by incorrectly rejecting the null hypothesis. It is typically defined as α and represents the threshold for considering a result as statistically significant. It is 5% for our case. 

- **Power Equation:** It is an equation that relates the sample size, statistical power, significance level and effect size. It is used to determine the sample size necessary to achieve a given statistical power under certain conditions.

#### Effect Size for two proportions/averages

This test must be performed using scalar values, such as propotions and average values.

In [45]:
# effect size parameters
p1 = 0.13
p2 = 0.15

# effect size
effect_size = sms.proportion_effectsize(p1, p2)

# statistical power
statistical_power = 0.8

In [46]:
effect_size

-0.0576728617308947

We may also use the method below:

In [50]:
def cohen_d(p1, p2):
    """
    Calculate Cohen's d effect size for two proportions.
    
    Parameters:
        p1 (float): Proportion for group 1.
        p2 (float): Proportion for group 2.
    
    Returns:
        float: Cohen's d effect size.
    """
    return (p1 - p2) / np.sqrt((p1 * (1 - p1) + p2 * (1 - p2)) / 2)

# p1 and p2 values
p1 = 0.13
p2 = 0.15

# Cohen's d
cohen_d_value = cohen_d(p1, p2)
print("Cohen's d:", cohen_d_value)


Cohen's d: -0.05766299317048917


In [53]:
def glass_delta(p1, p2):
    """
    Calculate Glass's delta effect size for two proportions.
    
    Parameters:
        p1 (float): Proportion for group 1.
        p2 (float): Proportion for group 2.
    
    Returns:
        float: Glass's delta effect size.
    """
    return (p1 - p2) / np.sqrt(p2 * (1 - p2))

def hedges_g(p1, p2, n1, n2):
    """
    Calculate Hedge's g effect size for two proportions.
    
    Parameters:
        p1 (float): Proportion for group 1.
        p2 (float): Proportion for group 2.
        n1 (int): Sample size for group 1.
        n2 (int): Sample size for group 2.
    
    Returns:
        float: Hedge's g effect size.
    """
    d = (p1 - p2) / np.sqrt((p1 * (1 - p1) + p2 * (1 - p2)) / 2)
    return d * (1 - 3 / (4 * (n1 + n2 - 9)))

def cohen_h(p1, p2):
    """
    Calculate Cohen's h effect size for two proportions.
    
    Parameters:
        p1 (float): Proportion for group 1.
        p2 (float): Proportion for group 2.
    
    Returns:
        float: Cohen's h effect size.
    """
    return (2 * np.arcsin(np.sqrt(p1)) - 2 * np.arcsin(np.sqrt(p2))) / np.pi

# p1 and p2 values
p1 = 0.13
p2 = 0.15

# sample size for Hedge's g :: calculated in the next section
n1 = 36308
n2 = 36308

# effect size
glass_delta_value = glass_delta(p1, p2)
hedges_g_value = hedges_g(p1, p2, n1, n2)
cohen_h_value = cohen_h(p1, p2)

print("Glass's delta:", glass_delta_value)
print("Hedge's g:", hedges_g_value)
print("Cohen's h:", cohen_h_value)


Glass's delta: -0.056011203361120365
Hedge's g: -0.0576623975358413
Cohen's h: -0.018357842053454587


#### Effect Size for two-dimensional numpy arrays

This test must be performed using two-dimensional arrays. 

In [54]:
# # Cohen's d
# pg.compute_effsize(x1, x2, eftype="cohen")

# # Glass's delta
# pg.compute_effsize(x1, x2, eftype="glass")

# # Hedge's g
# pg.compute_effsize(x1, x2, eftype="hedges")

# # Cohen's h
# sms.proportion_effectsize(np.mean(x1), np.mean(x2))

#### Sample Size

In [10]:
#sample size
sample_n = sms.NormalIndPower().solve_power(
                effect_size=effect_size,
                power=statistical_power,
                alpha=significance_level
)

In [11]:
sample_n

4719.474057599816

In [12]:
np.round(sample_n, 0).astype(int)

4719

In [13]:
math.ceil(sample_n)

4720

In [14]:
sample_n = math.ceil(sample_n)

In [15]:
print(f"The sample size of the control group is: {sample_n}.")
print(f"The sample size of the treatment group is: {sample_n}.")
print(f"The total sample size is: {2*sample_n}.")

The sample size of the control group is: 4720.
The sample size of the treatment group is: 4720.
The total sample size is: 9440.


Since we have the current conversion equals to 13% (4720), this value correspond to the size of each group. The total amount of people we must include in the test is:

In [16]:
conversion_rate = 0.13
total = sample_n/conversion_rate
total = math.ceil(total)
total

36308

In [17]:
print(f"The total people who were tested in control group was: {total}.")
print(f"The total people who were tested in treatment group was: {total}.")
print(f"The total people who were tested in both of the groups was: {2*total}.")

The total people who were tested in control group was: 36308.
The total people who were tested in treatment group was: 36308.
The total people who were tested in both of the groups was: 72616.


In [18]:
print(f"We have {data['user_id'].nunique()} people in our database.")

We have 290584 people in our database.


#### Number of samples per proportion difference